In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [2]:
tickers_map = {
    # Energy
    'CL': 'CL=F',   # WTI Crude Oil
    'NG': 'NG=F',   # Natural Gas
    'RB': 'RB=F',   # Gasoline
    'HO': 'HO=F',   # Heating Oil
    
    # Metals
    'GC': 'GC=F',   # Gold
    'SI': 'SI=F',   # Silver
    'HG': 'HG=F',   # Copper
    'PL': 'PL=F',   # Platinum
    
    # Agriculture
    'ZC': 'ZC=F',   # Corn
    'ZW': 'ZW=F',   # Wheat
    'ZS': 'ZS=F',   # Soybeans
    'KC': 'KC=F',   # Coffee
    'SB': 'SB=F',   # Sugar
    'CT': 'CT=F',   # Cotton
    
    # Livestock
    'LE': 'LE=F',   # Live Cattle
    'HE': 'HE=F',   # Lean Hogs
}

In [37]:
def create_trades_with_sizing(df_with_signals, initial_capital=1.0, position_size=0.52):
    """
    Version avec gestion de capital constante
    """
    trades_list = []
    position = 0
    entry_idx = None
    trade_num = 0
    capital = initial_capital
    
    df = df_with_signals.with_row_count('idx')
    
    for row in df.iter_rows(named=True):
        current_signal = row['signal']
        
        # Changement de position
        if current_signal != position:
            # Ferme position existante
            if position != 0 and entry_idx is not None:
                exit_price = row['close']
                entry_row = df.filter(pl.col('idx') == entry_idx).to_dicts()[0]
                entry_price = entry_row['close']
                
                # Calcul PnL
                price_change = (exit_price - entry_price) / entry_price
                pnl = price_change * position * position_size
                capital += pnl
                
                # Trade de sortie
                trades_list.append({
                    'timestamp': row['datetime'],
                    'position_number': trade_num,
                    'action': 'SELL' if position == 1 else 'BUY',
                    'price': exit_price,
                    'quantity_usd': position_size,
                    'position_size': position_size,
                    'pnl': pnl,
                    'cumulative_capital': capital
                })
                trade_num += 1
            
            # Ouvre nouvelle position si signal non-zero
            if current_signal != 0:
                trades_list.append({
                    'timestamp': row['datetime'],
                    'position_number': trade_num,
                    'action': 'BUY' if current_signal == 1 else 'SELL',
                    'price': row['close'],
                    'quantity_usd': position_size,
                    'position_size': position_size,
                    'pnl': 0.0,
                    'cumulative_capital': capital
                })
                position = current_signal
                entry_idx = row['idx']
    
    return pl.DataFrame(trades_list)

In [38]:
def calculate_performance_metrics(df_with_signals, df_trades=None):
    """
    Calcule Sharpe ratio, total return et nombre de trades
    
    Parameters:
    -----------
    df_with_signals : polars DataFrame
        DataFrame avec colonnes: timestamp, close, signal, strategy_returns
    df_trades : polars DataFrame (optional)
        DataFrame des trades. Si None, sera calculé automatiquement
        
    Returns:
    --------
    dict avec les métriques
    """
    # 1. Nombre de trades (changements de signal)
    if df_trades is None:
        # Compte les changements de signal (quand diff != 0)
        signal_changes = df_with_signals['signal'].diff().abs()
        num_trades = signal_changes.filter(signal_changes > 0).len()
    else:
        num_trades = len(df_trades)
    
    # 2. Strategy returns
    strategy_rets = df_with_signals['strategy_returns'].drop_nulls()
    
    if len(strategy_rets) == 0:
        return {
            'sharpe_ratio': np.nan,
            'total_return': np.nan,
            'annualized_return': np.nan,
            'annualized_volatility': np.nan,
            'num_trades': num_trades,
            'max_drawdown': np.nan,
            'win_rate': np.nan
        }
    
    # 3. Sharpe Ratio
    mean_return = strategy_rets.mean()
    std_return = strategy_rets.std()
    sharpe = (mean_return / std_return * np.sqrt(252)) if std_return > 0 else 0
    
    # 4. Total Return (cumulative)
    cumulative_returns = (1 + strategy_rets).product()
    total_return = cumulative_returns - 1
    
    # 5. Annualized Return
    num_years = len(strategy_rets) / 252  # Approximation
    if num_years > 0:
        annualized_return = (1 + total_return) ** (1 / num_years) - 1
    else:
        annualized_return = 0
    
    # 6. Annualized Volatility
    annualized_vol = std_return * np.sqrt(252)
    
    # 7. Max Drawdown
    cumulative = (1 + df_with_signals['strategy_returns'].fill_null(0)).cum_prod()
    running_max = cumulative.cum_max()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min()
    
    # 8. Win Rate (% de returns positifs)
    positive_returns = strategy_rets.filter(strategy_rets > 0).len()
    win_rate = positive_returns / len(strategy_rets) if len(strategy_rets) > 0 else 0
    
    return {
        'sharpe_ratio': sharpe,
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_vol,
        'num_trades': num_trades,
        'max_drawdown': abs(max_dd),
        'win_rate': win_rate,
        'num_periods': len(strategy_rets)
    }


def print_performance_summary(metrics):
    """
    Affiche un résumé propre des performances
    """
    print("\n" + "="*60)
    print("           PERFORMANCE SUMMARY")
    print("="*60)
    print(f"Sharpe Ratio:              {metrics['sharpe_ratio']:>10.2f}")
    print(f"Total Return:              {metrics['total_return']:>10.2%}")
    print(f"Annualized Return:         {metrics['annualized_return']:>10.2%}")
    print(f"Annualized Volatility:     {metrics['annualized_volatility']:>10.2%}")
    print(f"Max Drawdown:              {metrics['max_drawdown']:>10.2%}")
    print(f"Win Rate:                  {metrics['win_rate']:>10.2%}")
    print(f"Number of Trades:          {metrics['num_trades']:>10.0f}")
    print(f"Trading Periods:           {metrics['num_periods']:>10.0f}")
    print("="*60 + "\n")

In [39]:
from quanta.clients.yfinance import YahooFinanceClient
from datetime import datetime, timedelta

n=365
from_date = datetime.now() - timedelta(days=n)
to_date = datetime.now() - timedelta(days=1)

yh = YahooFinanceClient()
df_cl = yh.get_price(
    "CL=F",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval="1h", 
    postclean=True
)
df_cl

timestamp,datetime,open,high,low,close,volume
i64,datetime[μs],f64,f64,f64,f64,i64
1731884400,2024-11-18 00:00:00,66.860001,66.940002,66.610001,66.769997,1369
1731888000,2024-11-18 01:00:00,66.769997,67.110001,66.75,67.089996,1457
1731891600,2024-11-18 02:00:00,67.099998,67.129997,66.860001,67.029999,797
1731895200,2024-11-18 03:00:00,67.040001,67.160004,66.970001,67.120003,502
1731898800,2024-11-18 04:00:00,67.139999,67.32,67.129997,67.279999,625
…,…,…,…,…,…,…
1763139600,2025-11-14 18:00:00,60.16,60.299999,60.0,60.200001,19809
1763143200,2025-11-14 19:00:00,60.189999,60.279999,60.060001,60.07,13541
1763146800,2025-11-14 20:00:00,60.07,60.169998,59.939999,60.080002,26302


In [46]:
import polars as pl
import numpy as np

def momentum_strategy(df, lookback_period=252):
    # 1. Calcule returns 12-month (252 jours de trading)
    df = df.with_columns([
        (pl.col('close').pct_change(lookback_period)).alias('returns_12m')
    ])

    # 2. Signal simple (+1 si positif, -1 si négatif)
    df = df.with_columns([
        pl.when(pl.col('returns_12m') > 0).then(1)
        .when(pl.col('returns_12m') < 0).then(-1)
        .otherwise(0)
        .alias('signal')
    ])


    # 3. Daily returns
    df = df.with_columns([
        pl.col('close').pct_change().alias('daily_returns')
    ])

    # 4. Strategy returns (signal décalé d'un jour)
    df = df.with_columns([
        (pl.col('signal').shift(1) * pl.col('daily_returns')).alias('strategy_returns')
    ])
    return df

df_cl = momentum_strategy(df_cl, lookback_period=252)
df_cl


timestamp,datetime,open,high,low,close,volume,returns_12m,signal,daily_returns,strategy_returns
i64,datetime[μs],f64,f64,f64,f64,i64,f64,i32,f64,f64
1731884400,2024-11-18 00:00:00,66.860001,66.940002,66.610001,66.769997,1369,null,0,null,null
1731888000,2024-11-18 01:00:00,66.769997,67.110001,66.75,67.089996,1457,null,0,0.004793,0.0
1731891600,2024-11-18 02:00:00,67.099998,67.129997,66.860001,67.029999,797,null,0,-0.000894,-0.0
1731895200,2024-11-18 03:00:00,67.040001,67.160004,66.970001,67.120003,502,null,0,0.001343,0.0
1731898800,2024-11-18 04:00:00,67.139999,67.32,67.129997,67.279999,625,null,0,0.002384,0.0
…,…,…,…,…,…,…,…,…,…,…
1763139600,2025-11-14 18:00:00,60.16,60.299999,60.0,60.200001,19809,-0.008074,-1,0.000665,-0.000665
1763143200,2025-11-14 19:00:00,60.189999,60.279999,60.060001,60.07,13541,-0.009726,-1,-0.002159,0.002159
1763146800,2025-11-14 20:00:00,60.07,60.169998,59.939999,60.080002,26302,-0.006285,-1,0.000167,-0.000167


In [41]:
trades = create_trades_with_sizing(
    df_cl, 
    initial_capital=1.0, 
    position_size=0.519246)
trades

/tmp/ipykernel_688391/1518853554.py:11: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  df = df_with_signals.with_row_count('idx')


timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-12-05 03:00:00,0,"""BUY""",68.580002,0.519246,0.519246,0.0,1.0
2024-12-05 18:00:00,0,"""SELL""",68.239998,0.519246,0.519246,-0.002574,0.997426
2024-12-05 18:00:00,1,"""SELL""",68.239998,0.519246,0.519246,0.0,0.997426
2024-12-12 03:00:00,1,"""BUY""",70.260002,0.519246,0.519246,-0.01537,0.982055
2024-12-12 03:00:00,2,"""BUY""",70.260002,0.519246,0.519246,0.0,0.982055
…,…,…,…,…,…,…,…
2025-11-14 12:00:00,145,"""SELL""",59.560001,0.519246,0.519246,0.0,0.784443
2025-11-14 15:00:00,145,"""BUY""",60.040001,0.519246,0.519246,-0.004185,0.780259
2025-11-14 15:00:00,146,"""BUY""",60.040001,0.519246,0.519246,0.0,0.780259


In [42]:
metrics = calculate_performance_metrics(df_cl)
print_performance_summary(metrics)


           PERFORMANCE SUMMARY
Sharpe Ratio:                   -0.28
Total Return:                 -37.49%
Annualized Return:             -2.10%
Annualized Volatility:          6.85%
Max Drawdown:                  48.76%
Win Rate:                      45.63%
Number of Trades:                 148
Trading Periods:                 5582



In [43]:
df_cl

timestamp,datetime,open,high,low,close,volume,returns_12m,signal,daily_returns,strategy_returns
i64,datetime[μs],f64,f64,f64,f64,i64,f64,i32,f64,f64
1731884400,2024-11-18 00:00:00,66.860001,66.940002,66.610001,66.769997,1369,null,0,null,null
1731888000,2024-11-18 01:00:00,66.769997,67.110001,66.75,67.089996,1457,null,0,0.004793,0.0
1731891600,2024-11-18 02:00:00,67.099998,67.129997,66.860001,67.029999,797,null,0,-0.000894,-0.0
1731895200,2024-11-18 03:00:00,67.040001,67.160004,66.970001,67.120003,502,null,0,0.001343,0.0
1731898800,2024-11-18 04:00:00,67.139999,67.32,67.129997,67.279999,625,null,0,0.002384,0.0
…,…,…,…,…,…,…,…,…,…,…
1763139600,2025-11-14 18:00:00,60.16,60.299999,60.0,60.200001,19809,-0.008074,-1,0.000665,-0.000665
1763143200,2025-11-14 19:00:00,60.189999,60.279999,60.060001,60.07,13541,-0.009726,-1,-0.002159,0.002159
1763146800,2025-11-14 20:00:00,60.07,60.169998,59.939999,60.080002,26302,-0.006285,-1,0.000167,-0.000167


In [44]:
trades

timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-12-05 03:00:00,0,"""BUY""",68.580002,0.519246,0.519246,0.0,1.0
2024-12-05 18:00:00,0,"""SELL""",68.239998,0.519246,0.519246,-0.002574,0.997426
2024-12-05 18:00:00,1,"""SELL""",68.239998,0.519246,0.519246,0.0,0.997426
2024-12-12 03:00:00,1,"""BUY""",70.260002,0.519246,0.519246,-0.01537,0.982055
2024-12-12 03:00:00,2,"""BUY""",70.260002,0.519246,0.519246,0.0,0.982055
…,…,…,…,…,…,…,…
2025-11-14 12:00:00,145,"""SELL""",59.560001,0.519246,0.519246,0.0,0.784443
2025-11-14 15:00:00,145,"""BUY""",60.040001,0.519246,0.519246,-0.004185,0.780259
2025-11-14 15:00:00,146,"""BUY""",60.040001,0.519246,0.519246,0.0,0.780259


In [45]:
from quanta.clients.chart import ChartClient
chart_client = ChartClient()
chart_client.plot(
    df_cl, 
    "cl=F",  
    trades_df=trades, 
    theme='professional'
)

Plotting 5583 bars for cl=F with x_axis_type='row_nb'
With 295 trades
Debug: 295 trades after processing
shape: (5, 4)
┌─────────────────────┬─────────┬────────┬───────────┐
│ timestamp           ┆ x_value ┆ action ┆ price     │
│ ---                 ┆ ---     ┆ ---    ┆ ---       │
│ datetime[μs]        ┆ i64     ┆ str    ┆ f64       │
╞═════════════════════╪═════════╪════════╪═══════════╡
│ 2024-12-05 03:00:00 ┆ 252     ┆ BUY    ┆ 68.580002 │
│ 2024-12-05 18:00:00 ┆ 267     ┆ SELL   ┆ 68.239998 │
│ 2024-12-05 18:00:00 ┆ 267     ┆ SELL   ┆ 68.239998 │
│ 2024-12-12 03:00:00 ┆ 367     ┆ BUY    ┆ 70.260002 │
│ 2024-12-12 03:00:00 ┆ 367     ┆ BUY    ┆ 70.260002 │
└─────────────────────┴─────────┴────────┴───────────┘
